# 실습: 교차검증과 붓스트랩

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KoISLP/lab/blob/main/Ch05-resample-lab.ipynb)

이번 실습에서는 본문에서 다룬 재표집 기법들을 탐색해 보자. 몇몇 명령어는 개인 컴퓨터에서 실행하는 데 시간이 좀 걸릴 수 있다.

다시 한번, 여기 최상위 셀에서 라이브러리를 가져오는 것으로 시작한다.

In [ ]:
!pip install ISLP

In [ ]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split


이번 실습에서 사용할 새로운 라이브러리도 가져온다.

In [ ]:
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm


## 검증 세트 접근법
Auto 데이터 세트에 다양한 선형모형을 적합해서 얻은 테스트 오류율을 추정하기 위해 검증 세트(validation set) 방법을 사용한다.

데이터를 훈련 세트와 검증 세트로 나누기 위해 `train_test_split()` 함수를 사용한다. 관측값이 392개 있으므로 t`est_size=196` 인자를 사용해 크기가 196개인 세트 두 개로 동등하게 나눈다. 이처럼 무작위성을 포함하는 작업을 수행할 때는 나중에 결과를 똑같이 재현할 수 있도록 랜덤 시드를 설정하는 것이 좋다. 분할기의 랜덤 시드는 `random_state=0` 인자로 설정한다.

In [ ]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)


이제 훈련 세트 `Auto_train`에 해당하는 관측만 사용해 선형회귀를 적합할 수 있다.

In [ ]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()


이제 `results`의 `predict()` 메소드로 검증 데이터 세트에서 생성된 모형의 행렬을 평가한다. 또한 모형의 검증 MSE도 계산한다.

In [ ]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)


23.61661706966988

선형회귀 적합의 검증 MSE 추정값은 23.62이다.

고차 다항회귀의 검증 오차도 추정할 수 있다. 먼저 모형 문자열과 훈련 세트, 테스트 세트를 받아서 테스트 세트의 MSE값을 반환하는 함수 `evalMSE()`를 작성한다.

In [ ]:
def evalMSE(terms,
            response,
            train,
            test):

   mm = MS(terms)
   X_train = mm.fit_transform(train)
   y_train = train[response]

   X_test = mm.transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2)


이 함수로 선형, 이차, 삼차 적합을 사용한 다항회귀의 검증 MSE를 추정한다. 여기서 사용하는 `enumerate()` 함수는 for 루프를 반복할 때 객체의 값과 인덱스를 함께 제공한다.

In [ ]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE


array([23.61661707, 18.76303135, 18.79694163])

오류율은 각각 $23.62$, $18.76$, $18.80$이다. 다른 훈련/검증 분할을 대신 선택하면 검 증 세트에서 다소 다른 오류를 기대할 수 있다.

In [ ]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

훈련 세트와 검증 세트로 분할해 살펴보면 선형, 이차, 삼차항이 있는 모형의 검증세트 오류율이 각각 $20.76$, $16.95$, $16.97$로 나타난다.

앞서 알게 된 결과와 일치한다. `mpg`를 예측할 때 `horsepower`의 이차함수를 사용하는 모형이 `horsepower`의 선형함수만 포함하는 모형보다 성능이 더 좋지만 `horsepower`의 삼차함수를 사용했을 때 더 개선된다는 증거는 없다.

## 교차검증
이론적으로 모든 일반화선형모형에 대해 교차검증 추정값을 계산할 수 있다. 하지만 실제로 파이썬에서 교차검증을 수행하는 가장 간단한 방법은 `sklearn`을 사용하는 방법으로, GLM을 적합하기 위해 사용했던 `statsmodels`와는 다른 인터페이스 또는 API를 사용한다.

데이터 과학자들은 종종 다음과 같은 문제에 직면한다. “나에게는 작업 $A$를 수행하는 함수가 있고 그 함수를 작업 $B$를 수행하는 무언가에 집어넣어야 한다. 그래야 내 데이터 $D$에서 $B(AD)$를 계산할 수 있다”. $A$와 $B$가 자연스럽게 소통하지 않는다면 래퍼(wrapper)를 사용할 필요가 있다. `ISLP` 패키지에서는 `statsmodels`로 적합된 모형과 함께 `sklearn`의 교차검증 도구를 쉽게 사용할 수 있도록 `sklearn_sm()` 래퍼를 제공한다.

`sklearn_sm()` 클래스는 첫 번째 인자로 `statsmodels`에서 나온 모형을 받는다. 추가로 두 개의 선택적 인자를 받을 수 있다. `model_str`은 수식을 지정하는 데 사용하고 `model_args`는 모형을 적합할 때 사용하는 추가 인자의 사전(dictionary)이다. 예를 들어 로지스틱 회귀모형을 적합하려면 `family` 인자를 지정해야 한다. 이것은 `model_args={‘family’:sm.families.Binomial()}`로 전달된다.

여기 래퍼를 실제로 사용해 보자.

In [ ]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err


24.23151351792924

`cross_validate()` 함수는 적절한 `fit()`, `predict()`, `score()` 메소드가 있는 객체와 자질들의 배열인 `X` 그리고 반응변수 `Y`를 인자로 받는다. 또 `cross_validate()`는 `cv`라는 추가 인자도 포함하는데, 이 인자는 정수 $K$를 지정하면 $K$-겹 교차검증을 수행한다. 관측값 전체 개수에 해당하는 값을 제공하면 결과로 LOOCV가 나온다. `cross_validate()` 함수는 구성 요소가 여러 개인 사전(dictionary)을 만든다. 여기서 교차검증 테스트 점수(MSE)를 확인해 보면 24.23으로 추정된다.

점차 복잡한 다항 적합에 이 절차를 반복할 수 있다. 이 과정을 자동화하기 위해 이번에도 for 루프를 사용해 1~5차에 이르는 다항회귀를 반복적으로 적합하고, 교차검증 오류를 계산해 `cv_error` 벡터의 $i$번째 요소에 저장한다. for 루프의 변수 `d`는 다항식의 차수를 나타낸다. 벡터를 초기화하는 것으로 시작한다. 이 명령을 실행하는 데 몇 초 걸릴 수 있다.

In [ ]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

[그림 5.4]에서 볼 수 있듯이 선형과 이차 적합 사이에서 테스트 MSE 추정값이 급격 히 감소하지만 그 이상의 고차 다항식에서는 뚜렷한 개선이 없다.

앞에서 `outer()` 메소드와 `np.power()` 함수를 소개했다. `outer()` 메소드는 인자가 두 개인 연산에 적용된다. 예를 들어 `add()`, `min()`, `power()`와 같은 연산이다. 이 메소드는 두 배열을 인자로 받아 두 배열의 각 원소 쌍에 연산을 적용해 더 큰 배열을 형성한다.

In [ ]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)


array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

앞의 CV 예제에서는 $K = n$을 사용했지만 당연히 $K < n$을 사용할 수도 있다. 코드는 앞의 코드와 매우 유사하지만 훨씬 더 빠르다. 여기서는 `KFold()`로 데이터를 $K = 10$개의 랜덤 그룹으로 나눈다. `random_state`로 랜덤 시드를 설정하고 1~5차에 이르는 다항식 적합에 해당하는 CV 오류를 저장할 벡터 cv_error를 초기화한다.

In [ ]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0)  # 동일한 분할을 각 차수에서 사용
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

LOOCV에 비해 계산 시간이 훨씬 짧다는 점에 주목할 필요가 있다(원칙적으로 최소제곱 선형모형에 대한 LOOCV의 계산 시간은 $K$-겹 교차검증보다 빨라야 한다. LOOCV를 위한 식 (5.2)를 사용할 수 있기 때문이다. 하지만 제네릭(generic) 함수인 `cross_validate()`는 이 공식을 사용하지 않는다). 삼차 또는 그 이상의 다항식을 사용한다고 해서 단순한 이차 적합을 사용하는 것보다 테스트 오류를 낮춘다는 증거는 거의 찾아볼 수 없다.

`cross_validate()` 함수는 유연하여 다양한 분할 메커니즘을 인자로 받을 수 있 다. 예를 들어 `ShuffleSplit()` 함수를 사용하면 검증 세트 접근법을 K-겹 교차검증 만큼 쉽게 구현할 수 있다.

In [ ]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']


array([23.61661707])

다음을 실행해 테스트 오차의 변동을 추정할 수 있다.

In [ ]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()


(23.802232661034168, 1.421845094109185)

이 표준편차는 평균 테스트 점수나 개별 점수의 표집 변동(sampling variability)에 대한 유효한 추정값이 아니라는 점에 유의한다. 무작위로 선택된 훈련 표본들이 겹치면서 상관관계를 유발하기 때문이다. 그렇긴 하지만 서로 다른 무작위 분할(fold) 선택에서 발생하는 몬테카를로 변동이라는 발상을 제공한다.

## 붓스트랩
5.2절의 간단한 예제로 붓스트랩을 설명한다. `Auto` 데이터에서 선형회귀모형의 정확도를 추정하는 예제도 함께 볼 예정이다.

### 관심 통계량의 정확도 추정하기
붓스트랩 접근법의 가장 큰 장점은 거의 모든 상황에 적용할 수 있다는 점이다. 복잡한 수학적 계산을 요구하지 않는다. 파이썬에는 붓스트랩의 구현체가 여러 개 있지만, 표준오차를 추정하기 위해 붓스트랩을 사용하는 것은 매우 간단하기 때문에 여기서는 데이터가 데이터프레임에 저장되어 있는 경우에 대해 직접 작성한 함수를 사용한다.

붓스트랩을 설명하기 위해 간단한 예제로 시작해 보자. `ISLP` 패키지의 `Portfolio`데이터 세트는 5.2절에 설명되어 있다. 식 (5.7)에 주어진 모수 $\alpha$의 표본분산을 추 정하는 것이 목표다. `alpha_func()` 함수를 생성한다. 이 함수는 데이터프레임 `D`를 입력으로 받는데, 이 데이터프레임에 `X`와 `Y` 열이 있다고 가정한다. 이와 함께  $\alpha$를 추정할 때 어떤 관측값을 사용할지 알려 주는 벡터 `idx`도 받는다. 그런 다음 함수는 선택된 관측값에 기반해 $\alpha$의 추정값을 출력한다.

In [ ]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))


이 함수는 인자 `idx`로 인덱싱한 관측값에 최소 분산 공식(식 (5.7))을 적용해 $\alpha$의 추정값을 반환한다. 예를 들어 다음 명령어는 100개의 관측값을 모두 사용해 $\alpha$를 추정한다.

In [ ]:
alpha_func(Portfolio, range(100))

0.57583207459283

다음으로 `range(100)`에서 복원추출로 100개의 관측을 무작위로 선택한다. 이는 새로운 붓스트랩 데이터 세트를 구성하고 새 데이터 세트를 기반으로 $\hat{\alpha}$을 다시 계산 하는 것과 동치이다.

In [ ]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619004

이 과정을 일반화해 데이터프레임만 받는 임의의 함수에서 붓스트랩 표준오차를 계산하는 간단한 함수 `boot_SE()`를 작성할 수 있다.

In [ ]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

`for _ in range(B)`에서 루프 변수로 _를 사용한 점에 주의한다. 카운터의 값이 중요하지 않고 단지 루프가 B번 실행되기만 하면 충분할 때 자주 사용된다.

$B = 1,000$번 붓스트랩을 반복할 때 $\alpha$에 대한 추정값의 정확도를 평가하는 함수 를 사용해 보자

In [ ]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE


0.09118176521277699

최종 출력을 보면 ${\rm SE}(\hat{\alpha})$의 붓스트랩 추정값은 $0.0912$이다.

### 선형회귀모형의 정확도 추정
붓스트랩 접근법으로 통계적 학습 방법에서 나온 계수 추정값과 예측값의 변동을 평가할 수 있다. 여기서는 붓스트랩 접근법을 이용해 `Auto` 데이터 세트에서 `horsepower`를 사용해 `mpg`를 예측하는 선형회귀모형의 절편 $\beta_0$와 기울기 $\beta_1$에 대한 추정값의 변동을 평가한다. 붓스트랩을 사용해 얻은 추정값을 ${\rm SE}(\hat{\beta}_0)$과 ${\rm SE}(\hat{\beta}_1)$의 공식을 사용해 얻은 추정값과 비교한다. 후자는 3.1.2절에서 설명했다.

`boot_SE()` 함수를 사용하기 위해서는 데이터 프레임 `D`와 인덱스 `idx`만을 인자로 받는(첫 번째 인자인) 함수를 작성할 필요가 있다. 그러나 여기서는 모형식과 데이터를 지정한 특정 회귀 모델에 붓스트랩을 수행하려고 한다. 간단한 몇 단계를 살펴보자.

회귀모형을 붓스트랩하는 제네릭 함수 `boot_OLS()`를 작성한다. 공식(formula)을 받아서 그에 대응하는 회귀를 정의한다. `clone()` 함수를 사용해 새로운 데이터 프레임에 재적합할 수 있도록 공식(formula)의 복제본을 만든다. 어떤 파생 자질이라도, 예를 들어 `poly()`와 같이 정의된 것들도 (곧바로 보게 될 텐데) 재표집된 데이터 프레임에 재적합할 수 있다.

In [ ]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

이 함수는 `boot_SE()`의 첫 번째 인자로 딱 맞는 것은 아니다. 모형을 지정하는 첫 두 인자는 붓스트랩 과정에서 변경되지 않기 때문에 '동결'하는 것이 좋겠다. `functools` 모듈의 `partial()` 함수에서는 함수를 인자로 받아 왼쪽부터 시작해서 함수의 인자 중 일부를 동결한다. `boot_OLS()`의 첫 두 모형-수식 인자를 동결한다.

In [ ]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')


`hp_func?`을 입력하면 인자가 `D`와 `idx` 두 개임을 볼 수 있다. 첫 두 인자가 동결된 `boot_OLS()`의 한 버전이다. 따라서 `boot_SE()`의 첫 번째 인자로 딱 알맞다.

이제 `hp_func()` 함수를 사용해 절편과 기울기 항에 대한 붓스트랩 추정값을 만들 수 있다. 관측값 중에서 복원으로 랜덤하게 표본추출해 진행한다. 먼저 사용법을 보여 주는 차원에서 붓스트랩 표본 10개를 만들어 보자.

In [ ]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
          rng.choice(Auto.index,
                     392,
                     replace=True)) for _ in range(10)])


array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

다음으로, `boot_SE()` 함수를 사용해 절편과 기울기 항에 대한 1,000개의 붓스트랩 추정값의 표준오차를 계산한다.

In [ ]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se


intercept     0.731176
horsepower    0.006092
dtype: float64

 ${\rm SE}(\hat{\beta}_0)$의 붓스트랩 추정값은 0.85이고 ${\rm SE}(\hat{\beta}_1)$의 붓스트랩 추정값은 0.0074이다.
3.1.2절에서 논의했듯이 표준 공식을 이용해 선형모형에서 회귀계수의 표준오차를 계산할 수 있다. `ISLP.sm`의 `summarize()` 함수를 사용한다.

In [ ]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se


intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

3.1.2절의 공식을 이용해 $\hat{\beta}_0$와 $\hat{\beta}_1$의 표준오차 추정값을 구하면 절편은 0.717이고 기울기는 0.006이다. 흥미롭게도 이 결과는 붓스트랩으로 구한 추정값과 다소 차이가 있다. 이것이 붓스트랩에 문제가 있음을 나타내는 걸까? 실제로는 그 반대다. 000 쪽 식 (3,8)에서 제시된 표준 공식들은 특정 가정에 의존한다는 사실을 떠올려보자. 예를 들어 먼저 이 공식들은 미지의 모수 $\sigma^2$, 즉 소음 분산에 의존한다. 그런 다음 RSS를 사용해 $\sigma^2$을 추정한다. 이제 표준오차의 공식은 선형모형이 옳다는 가정에 의존하지 않지만 $\sigma^2$의 추정값에는 의존한다. 000 쪽 [그림 3.8]에서 볼 수 있듯이 데이터에 비선형 관계가 있으므로 선형 적합에서 나온 잔차들은 부풀려지고, 따라서 $\sigma^2$도 부풀려질 것이다. 다음으로 표준 공식들은 (다소 비현실적으로) $x_i$는 고정되어 있고 모든 변동은 오차 $\epsilon_i$에서 비롯된다고 가정한다. 붓스트랩 접근법은 이런 가정에 의존하지 않으므로 $\hat{\beta}_0$와 $\hat{\beta}_1$의 표준오차에 대해 `sm.OLS`보다 더 정확한 추정값을 낼 가능성이 높다.

다음에서는 붓스트랩 표준오차 추정값과 데이터에 이차 모형을 적합해서 얻은 표준 선형회귀 추정값을 계산한다. 이 모형은 데이터에 잘 적합했기 때문에[그림 3.8]) 이제 ${\rm SE}(\hat{\beta}_0)$, ${\rm SE}(\hat{\beta}_1)$, ${\rm SE}(\hat{\beta}_2)$의 붓스트랩 추정값과 표준 추정값은 더 유사하다.

In [ ]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)


intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

이 결과를 `sm.OLS()`를 이용해 계산한 표준오차와 비교해 보자.

In [ ]:
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']


intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64